In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip uninstall clr
!pip install pythonnet
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('corrected_labelled.csv')
df = df.dropna()

#target variable will nps split into demoters, passives and promoters
df['label'] = np.where(df['nps'] == 3,2,
                  np.where(df['nps'] == 1,0,1))

gb = df.groupby('label')['label'].count()
print(gb)

df.info()

label
0    411
1    159
2    565
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   response       1135 non-null   object
 1   nps            1135 non-null   int64 
 2   Term           1135 non-null   object
 3   ticket_number  1135 non-null   object
 4   grp            1135 non-null   object
 5   label          1135 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 53.3+ KB


In [ ]:
import re
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()

df['response'] = df['response'].apply(lambda x: text_preprocessing(x))
df['Term'] = df['Term'].apply(lambda x: text_preprocessing(x))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df[['response','Term']],df['label'], test_size = 0.33, random_state=42)
X = df[['response','Term']].values.tolist()
y = df['label'].to_list()
# #calculating class weights
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight = 'balanced', classes = [2,1,0], y = y_train)
type(class_weights)

numpy.ndarray

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

Creating DatasetDicts to be able to create tensors

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

train = X_train
train['label'] = y_train.to_list()

test = X_val
test['label'] = y_val.to_list()

# train_ds = Dataset.from_pandas(train)
# val_ds = Dataset.from_pandas(val)

# ds = DatasetDict()

# ds['train'] = train_ds
# ds['validation'] = val_ds
# ds

In [ ]:
# import datasets
# from datasets import Dataset, DatasetDict
# X = df[['response','Term']]
# y = df['label'].to_list()

# ds = Dataset.from_pandas(df[['response','Term','label']])


In [ ]:


def tokenize_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=500,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return TensorDataset(input_ids, attention_masks, labels)

In [ ]:
#Hugging face model checkpoint
checkpoint = "yangheng/deberta-v3-large-absa-v1.1"

Tokenising all datasets in the datasetdict

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = False)
def tokenize_function(example):
    return tokenizer(example["response"], example["Term"], truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/375 [00:00<?, ? examples/s]

Removing unneeded columns and renaming others to ensure compatibality

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["response", "Term", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
#creating the dataloader to set the batches
from torch.utils.data import DataLoader
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
# Move the model to GPU
model = model.to(device)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 50]),
 'token_type_ids': torch.Size([8, 50]),
 'attention_mask': torch.Size([8, 50])}

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding
def tokenize_data(texts1, texts2, labels, checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = False)

    input_ids = []
    attention_masks = []

    for text1, text2 in zip(texts1, texts2):
        encoding = tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=500,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return TensorDataset(input_ids, attention_masks, labels)

#ds = tokenize_data(df['response'].to_list(), df['Term'].to_list(), df['label'].to_list(), checkpoint)

In [ ]:
import datasets
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import datasets
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader


def tokenize_function(example):
    checkpoint = "yangheng/deberta-v3-large-absa-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = False)
    return tokenizer(example["response"], example["Term"], truncation=True)

def create_dataloader(df, checkpoint):
  ds = Dataset.from_pandas(df)

  tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast = False)
  tokenized_datasets = ds.map(tokenize_function, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  tokenized_datasets = tokenized_datasets.remove_columns(["response", "Term", "__index_level_0__"])
  tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
  tokenized_datasets.set_format("torch")




  dataloader = DataLoader(
      tokenized_datasets, shuffle=True, batch_size=8, collate_fn=data_collator
  )


  return dataloader


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, f1_score

# Define a custom scoring function (e.g., F1-score)
def custom_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')
def predicted_values(model, dataloader, device, class_weights):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_predicted_labels = []
    all_true_labels = []

    tensor_weights = torch.tensor(np.float32(class_weights)).to(device)
    loss_fn = nn.CrossEntropyLoss(weight = tensor_weights)

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            b_labels = batch['labels']
            b_input_ids = batch['input_ids']
            b_attn_mask = batch['attention_mask']

            logits = model(**batch)
            loss = loss_fn(logits['logits'], b_labels)

            total_loss += loss.item()

            # Calculate accuracy
            _, predicted_labels = torch.max(logits['logits'], 1)
            correct_predictions += (predicted_labels == b_labels).sum().item()
            total_samples += b_labels.size(0)

            all_predicted_labels.extend(predicted_labels.cpu().numpy())
            all_true_labels.extend(b_labels.cpu().numpy())

    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples

    # Calculate weighted F1 score
    #precision, recall, f1, _ = precision_recall_fscore_support(b_labels.cpu().numpy(), predicted_labels.cpu().numpy(), average='weighted')
    #print(b_labels.cpu().numpy())
    print('break')
    #print(predicted_labels.cpu().numpy())
    return  b_labels.cpu().numpy(), predicted_labels.cpu().numpy()

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch.nn as nn

def training_logic(model,train_dataloader,  lr, wdecay, epsilon, betas, warmup, epoch, class_weights, device, eval_dataloader):
  #downloading model checkpoint from huggingface
  # model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
  # model = model.to(device)

  #setting optimiser values
  optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wdecay, eps = epsilon, betas = betas)

  num_epochs = epoch
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps= warmup*num_training_steps,
      num_training_steps=num_training_steps,
  )

  tensor_weights = torch.tensor(np.float32(class_weights)).to(device)
  loss_fn = nn.CrossEntropyLoss(weight = tensor_weights)
  model.train()

  for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        b_labels = batch['labels']
        b_input_ids = batch['input_ids']
        b_attn_mask = batch['attention_mask']
        logits = model(**batch)
        loss = loss_fn(logits['logits'],b_labels)

        #validation loss
        # if ((step) % evaluate_every_n_steps == 0) or (step == len(train_dataloader) - 1):
        #   val_loss, val_accuracy, val_prec, val_recall, val_f1 = evaluate_model(model, eval_dataloader, loss_fn, device)
        #   print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
        #   # Optionally, you can store the loss and accuracy values for later plotting.
        #   # Add code for plotting as needed.
        #   epoch_list.append(epoch + 1)
        #   step_list.append(step)
        #   train_loss_list.append(loss.item())
        #   val_loss_list.append(val_loss)
        #   val_acc_list.append(val_accuracy)
        #   val_recall_list.append(val_recall)
        #   val_prec_list.append(val_prec)
        #   val_f1_list.append(val_f1)


        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

  y_true, y_pred = predict(model, eval_dataloader, loss_fn, device)
  y_true, y_pred = predicted_values(self.model, eval_dataloader, device, class_weights)

  return y_true, y_pred







In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, checkpoint, num_labels, lr=5e-5, wdecay=0.01, epsilon=1e-8, betas=(0.9, 0.999), warmup=0, epochs=3, class_weights=None):
        self.checkpoint = checkpoint
        self.num_labels = num_labels
        self.lr = lr
        self.wdecay = wdecay
        self.epsilon = epsilon
        self.betas = betas
        self.warmup = warmup
        self.epochs = epochs
        self.class_weights = class_weights
        self.model = None  # To be initialized during training

    def fit(self, X, y):
        train_dataloader = create_dataloader( train, self.checkpoint )
        eval_dataloader = create_dataloader( test, self.checkpoint )
        self.model = AutoModelForSequenceClassification.from_pretrained(self.checkpoint, num_labels=self.num_labels)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(device)

        # Training logic
        y_true, y_pred = training_logic(self.model,
            train_dataloader,  self.lr, self.wdecay, self.epsilon,
            self.betas, self.warmup, self.epochs, self.class_weights, device, eval_dataloader
        )

        return self

    def predict(self, X):
        eval_dataloader = create_dataloader( test, self.checkpoint )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Prediction logic
        y_true, y_pred = predicted_values(self.model, dataloader, device, self.class_weights)

        correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
        total_samples = len(y_true)
        accuracy = correct_predictions / total_samples

        print(f'test accuracy: {accuracy}')

        return y_pred

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import torch
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)


param_grid = {
    'lr': [5e-5, 3e-5, 2e-5],
    'wdecay': [0.01, 0.001, 0.0001],
    'epsilon': [1e-8, 1e-7, 1e-9],
    'betas': [(0.9, 0.999), (0.95, 0.98), (0.8, 0.9)],
    'warmup': [0.01, 0.05, 0.1],  # Adjust according to the total number of training steps
    'epochs': [2, 3, 4],
    'class_weights': [class_weights]

}


param_grid = {
    'lr': [5e-5],
    'class_weights': [class_weights]
}
#Hugging face model checkpoint
checkpoint = "yangheng/deberta-v3-large-absa-v1.1"

custom_scorer = make_scorer(f1_score, average='weighted')
custom_estimator = CustomEstimator(checkpoint, 3)
# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=custom_estimator,
    param_grid=param_grid,
    cv=2,
    scoring=custom_scorer,
    n_jobs=1,
    verbose=2
)

# Fit the grid search to the data
grid_search.fit(X_train,y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


Map:   0%|          | 0/760 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/375 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
ds.tensors[0]

tensor([[   1,  584,  428,  ...,    0,    0,    0],
        [   1,  278,  284,  ...,    0,    0,    0],
        [   1,  278,  284,  ...,    0,    0,    0],
        ...,
        [   1, 2444,  284,  ...,    0,    0,    0],
        [   1,  426,  772,  ...,    0,    0,    0],
        [   1,  426,  772,  ...,    0,    0,    0]])